In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

## Day 49 Lecture 1 Assignment

In this assignment, we will apply GMM (Gaussian Mixture Modeling) clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [2]:
%matplotlib inline

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.special import entr

<IPython.core.display.Javascript object>

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [4]:
# answer goes here
nba = pd.read_csv('data/nba_player_seasons.csv')
nba.head()




,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2019,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,...,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,2019,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,...,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,2019,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,...,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,...,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,...,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9


<IPython.core.display.Javascript object>

The goal is to cluster these player-seasons to identify potential player "archetypes".  
The pre-processing steps will be identical to what we previously did for K-means.

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

In [5]:
# answer goes here

nba = nba[nba['GS']>=20]
nba=nba[nba['MP']>=10]



<IPython.core.display.Javascript object>

Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [6]:
# answer goes here

numeric_cols = ['FG', 'FG%', 'FT%', 'ORB', 'DRB', 'AST', 'STL', 'BLK', 'PTS', 'MP']

scaler = StandardScaler() 
nba_final = scaler.fit_transform(nba[numeric_cols])




<IPython.core.display.Javascript object>

Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

Run both K-Means and Gaussian mixtures modeling twice, once with 3 cluster and once with 7 clusters. Print out the resulting means for all 4 scenarios (KM+3, GMM+3, KM+7, GMM+7). When printing the means, transform the scaled versions back into their corresponding unscaled values. 

What "archetypes" do you see? Are the archetypes identified by GMM similar to those identified by K-Means? How do the means of GMM differ from those of K-Means?

In [7]:
# answer goes here

gmm_3 = GaussianMixture(n_components=3)
gmm_7 = GaussianMixture(n_components=7)

km_3 = KMeans(n_clusters = 3)
km_7 = KMeans(n_clusters = 7)

gmm_3.fit(nba_final)
gmm_7.fit(nba_final)
km_3.fit(nba_final)
km_7.fit(nba_final)


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=7, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

<IPython.core.display.Javascript object>

In [29]:
gmm3_means_df = pd.DataFrame(
    scaler.inverse_transform(gmm_3.means_), columns=numeric_cols
)
gmm3_means_df.style.background_gradient()

,FG,FG%,FT%,ORB,DRB,AST,STL,BLK,PTS,MP
0,4.300389,0.522804,0.691041,2.127430,5.014420,1.486263,0.718790,0.993952,10.843922,25.330495
1,4.006488,0.430836,0.775879,0.600631,2.910307,2.697561,0.869831,0.298403,10.927237,26.856198
2,6.878374,0.456176,0.813523,1.089096,4.908400,4.710257,1.217177,0.597897,19.300985,33.124155


<IPython.core.display.Javascript object>

In [30]:
pd.Series(gmm_3.predict(nba_final)).value_counts()

1    350
0    234
2    220
dtype: int64

<IPython.core.display.Javascript object>

In [31]:
km3_means_df = pd.DataFrame(
    scaler.inverse_transform(km_3.cluster_centers_), columns=numeric_cols
)
km3_means_df.style.background_gradient()

,FG,FG%,FT%,ORB,DRB,AST,STL,BLK,PTS,MP
0,7.094515,0.458675,0.810338,1.056118,4.768354,4.780591,1.265823,0.600000,19.818143,33.521097
1,3.697128,0.434209,0.771292,0.703394,3.006005,2.349608,0.808877,0.337076,10.071802,25.730287
2,4.554348,0.540190,0.674837,2.409783,5.526087,1.588043,0.709783,1.111957,11.332065,26.225543


<IPython.core.display.Javascript object>

In [32]:
pd.Series(km_3.predict(nba_final)).value_counts()

1    383
0    237
2    184
dtype: int64

<IPython.core.display.Javascript object>

In [34]:
gmm7_means_df = pd.DataFrame(
    scaler.inverse_transform(gmm_7.means_), columns=numeric_cols
)
gmm7_means_df.style.background_gradient()

,FG,FG%,FT%,ORB,DRB,AST,STL,BLK,PTS,MP
0,3.473713,0.507136,0.687574,1.730259,4.170753,1.367204,0.730185,0.720442,8.678872,22.997906
1,4.580110,0.429159,0.760163,0.694810,3.435435,4.258602,1.181205,0.380299,12.394958,29.227741
2,6.868260,0.494737,0.768690,1.884970,6.411051,3.575432,0.917768,1.329817,18.243719,31.643352
3,3.367707,0.424219,0.783393,0.473758,2.351699,2.289890,0.770754,0.240085,9.260839,24.851356
4,4.893472,0.597725,0.594857,3.575301,7.430139,1.232298,0.752110,1.489878,12.142820,29.140253
5,5.194413,0.446081,0.806522,0.937764,4.141283,1.923006,0.806676,0.398678,14.269678,30.001016
6,7.314555,0.447000,0.836343,0.781300,3.918894,5.645765,1.333318,0.375132,20.967214,33.854389


<IPython.core.display.Javascript object>

In [36]:
pd.Series(gmm_7.predict(nba_final)).value_counts()

0    172
3    161
5    124
6    117
1     97
2     96
4     37
dtype: int64

<IPython.core.display.Javascript object>

In [35]:
km7_means_df = pd.DataFrame(
    scaler.inverse_transform(km_7.cluster_centers_), columns=numeric_cols
)
km7_means_df.style.background_gradient()

,FG,FG%,FT%,ORB,DRB,AST,STL,BLK,PTS,MP
0,4.604211,0.434284,0.747495,0.836842,3.650526,4.923158,1.416842,0.433684,12.483158,30.458947
1,8.079000,0.457660,0.831570,0.879000,4.369000,5.793000,1.371000,0.442000,23.113000,34.604000
2,2.722941,0.427294,0.749282,0.628824,2.554118,1.798235,0.691765,0.331765,7.389412,22.348235
3,3.620492,0.536697,0.670648,2.083607,4.546721,1.414754,0.628689,0.863934,8.962295,23.150000
4,7.047059,0.492435,0.782871,2.020000,6.728235,3.027059,0.927059,1.330588,18.684706,31.971765
5,5.396667,0.604600,0.576400,3.703333,8.156667,1.290000,0.866667,1.623333,13.303333,30.340000
6,5.072277,0.441921,0.813975,0.751485,3.509901,2.494059,0.839604,0.331188,13.888119,29.799010


<IPython.core.display.Javascript object>

In [37]:
pd.Series(km_7.predict(nba_final)).value_counts()

6    202
2    170
3    122
1    100
0     95
4     85
5     30
dtype: int64

<IPython.core.display.Javascript object>

Predict the likelihood of each player belonging to one of the 3 clusters using the GMM model. Then, calculate the entropy for each set of predicted probabilities. 

We will use entropy as a measure of how confident we are in the predicted class label. If we had no confidence in our prediction, we would assign 33% probability to each class, while if we were totally confident, we would assign 100% to one class. Entropy would be at a maximum in the "no confidence" scenario and a minimum in the "full confidence" scenario, which makes it a reasonable way to quantify our uncertainty in our prediction. There are certainly other methods as well; feel free to experiment with them if desired.

Which five predicted labels are we least confident about? Which five are we most confident about? Print out the associated details (season, player name, stats, etc.) from those players.

In [54]:
from scipy.stats import entropy

<IPython.core.display.Javascript object>

In [60]:
pd.Series(entropy(gmm_3.predict_proba(nba_final), axis=1)).sort_values(ascending=False)

560    1.085219e+00
403    1.016655e+00
593    9.029810e-01
207    8.858819e-01
293    8.839588e-01
           ...     
713    4.268353e-27
603    9.528053e-31
282    1.228307e-35
85     3.690737e-45
481    1.440818e-45
Length: 804, dtype: float64

<IPython.core.display.Javascript object>

In [40]:
# answer goes here

prob_df = pd.DataFrame(gmm_3.predict_proba(nba_final))
prob_df


,0,1,2
0,1.000000e+00,1.577570e-54,1.733700e-21
1,1.000000e+00,2.708081e-08,5.255024e-12
2,9.999266e-01,1.007888e-25,7.338485e-05
3,1.000000e+00,3.620775e-20,2.287732e-11
4,1.246332e-01,2.009009e-02,8.552767e-01
...,...,...,...
799,3.140220e-13,8.222532e-01,1.777468e-01
800,4.049758e-09,4.073066e-06,9.999959e-01
801,8.073641e-01,1.476441e-04,1.924883e-01
802,3.592834e-01,1.125730e-10,6.407166e-01


<IPython.core.display.Javascript object>

In [51]:
uncertain_idxs = (
    pd.DataFrame(entr(gmm_3.predict_proba(nba_final)))
    .mean(axis=1)
    .sort_values(ascending=False)[:5]
    .index
)

<IPython.core.display.Javascript object>

In [61]:
uncertain_idxs

Int64Index([560, 403, 593, 207, 293], dtype='int64')

<IPython.core.display.Javascript object>

In [53]:
prob_df.loc[uncertain_idxs]

,0,1,2
560,0.270978,0.404199,0.324823
403,0.258741,0.527884,0.213376
593,0.168435,0.192207,0.639358
207,0.080291,0.340962,0.578747
293,0.641460,0.124292,0.234248


<IPython.core.display.Javascript object>